# Agrupamento das linhas de operação associadas com acidentes

### Importando bibliotecas

In [ ]:
import pandas as pd
from resolve_path import ajuste_path

### Lê o dataset com a associação entre Operações e Acidentes já feita

In [ ]:
pathUtil = ajuste_path('data/util/')
df = pd.read_csv(pathUtil + 'dataset_associado.csv', encoding='utf-8')

#### Guarda a data do acidente por conta do dia do acontecido que precisa ser readicionado posteriormente

In [ ]:
df['data_acidente_copia'] = df['data_acidente']

### Passa a data para datetime

In [ ]:
df['data_inicio'] = pd.to_datetime(df['data_inicio'])
df['data_fim'] = pd.to_datetime(df['data_fim'])

## Ideia para agrupar as linhas de operação
- Objetivo: reduzir o desbalanço entre linhas com e sem acidente.
- Ideia: agrupar o trabalho feito por um funcionario em um mes em cada local
- Caminho tomado: 
  - Criar novas linhas de operação para todas as operações que duram mais de um mês, e fazer com que cada linha represente um dos meses em que a operação esteve ativa; 
  - A partir do momento que todas as linhas representam um mês, pode-se usar um groupby com as chaves primárias sendo o conjunto [ano, mês, local de instalação, ID pessoal];
  - No groupby, é necessário especificar como quer se agrupar cada coluna, as de HH fazem mais sentido somar, enquanto as de acidente faz mais sentido escolher a informação completa do primeiro acidente, pois é muito improvável que aconteça mais de um acidente com um mesma pessoa em um mesmo mês em um mesmo local;
  

### Divide a quantidade total de trabalho feita em uma operação para todos os meses em que ela esteve ativa, dividido igualmente

In [ ]:
# conta a quantidade de meses em que a operação esteve ativa
df["meses"] = (
    (df["data_fim"].dt.year - df["data_inicio"].dt.year) * 12
    + (df["data_fim"].dt.month - df["data_inicio"].dt.month)
) + 1
df["hh_por_mes"] = df["hh"] / df["meses"]

### Expansão das linhas de Operação para cada um dos meses em que esteve ativa

Nesta seção, é criada uma função 
para cada linha do df que tenha a coluna meses > 1 são criadas novas linhas para representar cada mes.

Exemplo da aplicação da função:
|op         |data inicio    |data fim   |
|-----------|---------------|-----------|
|operacao1  |2022-02        |2022-04    |

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&darr;

|op         |ano_mes    |
|-----------|-----------|
|operacao1  |2022-02    |
|operacao1  |2022-03    |
|operacao1  |2022-04    |

In [ ]:
def expande_df(df):
    '''
    Para cada linha do df que tenha a coluna meses > 1
    são criadas novas linhas para representar cada mes.

    Exemplo:
    |op         |data inicio    |data fim   |       |op         |ano_mes    |
                                                    |operacao1  |2022-02    |
    |operacao1  |2022-02        |2022-04    |   =>  |operacao1  |2022-03    |
                                                    |operacao1  |2022-04    |
    '''
    # Gerar o espaço de tempo
    espaco_de_tempo = pd.date_range(
        start='2020-01-01', end=pd.Timestamp.now(), freq="MS")
    espaco_de_tempo = pd.DataFrame(
        {'merge': [1]*len(espaco_de_tempo), 'ano_mes': espaco_de_tempo})
    # Marcar operações que duram mais de um mês
    df.loc[df['meses'] > 1, 'dura_mais_de_mes'] = 1

    # Dropando colunas desnecessárias
    colunas_a_dropar = ["hh"]
    df = df.drop(columns=colunas_a_dropar)

    # Expansão das operações que duram mais de um mês para todos os meses,
    # mesmo os que não estava ativa a operação
    df_expandido = pd.merge(df, espaco_de_tempo, left_on='dura_mais_de_mes',
                            right_on='merge', how='left').drop(columns=['merge'])

    # Tratamento das colunas de data para garantir o correto agrupamento
    df_expandido['dura_mais_de_mes'] = df_expandido['dura_mais_de_mes'].fillna(
        0)
    df_expandido.loc[df_expandido['dura_mais_de_mes']
                     == 0, 'ano_mes'] = df_expandido['data_fim']
    df_expandido['ano_mes'] = df_expandido['ano_mes'].dt.to_period('M')
    df_expandido['data_inicio'] = df_expandido['data_inicio'].dt.to_period('M')
    df_expandido['data_fim'] = df_expandido['data_fim'].dt.to_period('M')
    df_expandido['data_acidente'] = pd.to_datetime(
        df_expandido['data_acidente'])
    df_expandido['data_acidente'] = df_expandido['data_acidente'].dt.to_period(
        'M')

    # Eliminar linhas onde a operação não estava ativa
    df_expandido = df_expandido[(df_expandido['ano_mes'] >= df_expandido['data_inicio']) &
                                (df_expandido['ano_mes'] <= df_expandido['data_fim'])]
    return df_expandido

In [ ]:
df_expandido = expande_df(df)

### Faz a contagem do HH para cada categoria de txt.breve_operacao e cada categoria de denominacao_tam

Os valores do HH de cada tipo são adicionados em colunas novas, para que no agrupamento consiga-se manter esta informação em uma mesma linha;

Desta maneira, ao final do agrupamento se terá uma linha como por o seguinte exemplo:

|funcionário |ano    |mês|local de instalação    |HH total   |HH serviço |HH relatório   |HH PLANEJADA   |HH MELHORIA|
|------------|-------|---|-----------------------|-----------|-----------|---------------|---------------|-----------|
|1000001     |2023   |06 |S-S-ARE                |120        |80         |40             |120            |0          |

A soma de todo o HH das categorias de txt.breve_operacao será sempre igual ao HH total, assim como o HH de todas as categorias de denominacao_tam somadas.

In [ ]:
colunas_tipo_atividade = ["txt.breve_operacao", "denominacao_tam"]

# separar colunas de acidente apenas
lista_colunas_acidentes = df_expandido.columns[df_expandido.columns.str.contains(
    'acidente')].tolist()

# Colocar NAN nas colunas da acidente para as linhas em a data de acidente nao bate com a data da operação
df_expandido.loc[(df_expandido['data_acidente'] != pd.NA) & (
    df_expandido['ano_mes'] != df_expandido['data_acidente']), lista_colunas_acidentes] = pd.NA


# Para cada categoria de atividade, criar uma nova coluna para armazenar o HH dessa categoria
for coluna_tipo_atividade in colunas_tipo_atividade:
    for atividade in df_expandido[coluna_tipo_atividade].unique():
        # trata a exceção de haver categorias com o mesmo nome em colunas diferentes
        if "hh_de_" + atividade in df_expandido.columns:
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "hh_de_" + atividade + "_de_" + coluna_tipo_atividade] = df_expandido["hh_por_mes"]
        else:  # adiciona à sua propria coluna o hh desta categoria de atividade
            df_expandido.loc[df_expandido[coluna_tipo_atividade] == atividade,
                             "hh_de_" + atividade] = df_expandido["hh_por_mes"]

df_expandido.drop(columns=['txt.breve_operacao',
                  'tam', 'denominacao_tam'], inplace=True)

### Criação do dicionário para ser usado no groupby
Este dicionário especifica o que acontece com cada coluna do dataset, com algumas sendo a soma do total dos valores que aparecem na coluna, e outras sendo o primeiro valor pois só é suposto haver um valor para estas colunas em cada chave primária.

Atualmente apenas as colunas de HH precisam de um tratamento diferente de pegar apenas o primeiro valor.

In [ ]:
# Define dicionario, indicando qual operação será efetuada sobre quais colunas durante o agrupamento

# Retirando as colunas que são as variáveis do groupby da lista do dicionário
colunas = list(df_expandido.columns)
colunas.remove("no_pessoal")
colunas.remove("local_de_instalacao")
colunas.remove("ano_mes")

# Separando colunas de hh
colunas_hh = [
    col for col in colunas if col.startswith('hh')]

# Pegando apenas as colunas que nao sao de hh
colunas_sem_hh = [col for col in colunas if not col.startswith('hh')]

# Definindo operacoes que serao aplicadas sobre as colunas
dicionario_hh = {col: 'sum' for col in colunas_hh}
dicionario_hh.update({col: 'first' for col in colunas_sem_hh})

### Agrupamento por mês, ano, local e id como chave primária

In [ ]:
# Agrupando por ano_mes, local e no_pessoa, igualmente operando sobre as outras colunas
df_agrupado = df_expandido.groupby(
    ['ano_mes', 'local_de_instalacao', 'no_pessoal']).agg(dicionario_hh).reset_index()

# após o groupby a coluna hh_por_mes agora tem os valores das diferentes operações todas somadas
df_agrupado.rename(columns={'hh_por_mes': 'hh_total'}, inplace=True)

# Filtra para considerar apenas registros a partir de 2020-01
df_agrupado = df_agrupado[df_agrupado['ano_mes'] >= '2020-01']

### Acrescentar coluna para indicar se teve ou não acidente para facilitar a análise do modelo

In [ ]:
df_agrupado['acidente'] = pd.NA  # cria a coluna

df_agrupado.loc[df_agrupado['id_acidente'].notna(), 'acidente'] = 1
df_agrupado.loc[df_agrupado['id_acidente'].isna(), 'acidente'] = 0

### Restaura a coluna data do acidente para recuperar o dia em que ele ocorreu

In [ ]:
df_agrupado.drop(columns={'data_acidente'}, axis=1, inplace=True)
df_agrupado.rename(
    columns={'data_acidente_copia': 'data_acidente'}, inplace=True)

### Exportando dataset

In [ ]:
df_agrupado.to_csv(pathUtil + "dataset_treinamento.csv",
                   encoding='utf-8', index=False)